<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c9a826eb1daaebec642ccb6f689b854676084725768c44eb149884498a2d5b92
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:


      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-21 14:12:53
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.42 C
-------------------
Today PnL: 81.21 K (0.58%)
Current PnL: -19.62 L (-13.07%)
CY Booked + Current PnL: -7.72 L (-5.15%)
-------------------
Total profit:  2.35 L
Total loss:  -21.97 L
-------------------
Total Booked + Current PnL: 19.26 L (15.64%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.39%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 86.79 L (61.18%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.38%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1664.80,-14.64,74.0,X-LC,4.19,228078.0,23578.0,0.0,1.57,11.53,0.00,11.53,10.0,inf,1.62,21.51,X40N,ATH,PHARMA
56,RELIANCE,1533.00,-8.41,74.0,X-LC,2.87,229024.0,18658.0,10123.0,0.09,8.87,4.42,13.68,37.0,1.84,1.63,26.86,XY25,NTT,REFINERIES
83,VOLTAS,1530.00,4.10,66.0,X-MC,2.03,218445.0,26703.0,11053.0,0.80,13.93,5.06,19.69,99.0,2.42,1.55,21.67,XY25,NTT,AC
76,TTKPRESTIG,770.00,95.94,49.0,M-SC,1.84,84456.0,-16321.0,16418.0,1.14,-16.20,19.44,0.09,245.0,-0.99,0.60,10.96,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-13.61,64.0,X-MC,2.92,148414.0,12281.0,18849.0,-0.27,9.02,12.70,22.86,91.0,0.65,1.05,22.77,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-37.00,70.0,M-SC,4.00,106500.0,15002.0,63751.0,4.57,16.40,59.86,86.07,223.0,0.24,0.76,54.66,XR,NTT,DURABLES
14,BSOFT,836.99,-20.22,45.0,H-SC,15.45,89467.0,-51186.0,123965.0,3.51,-36.39,138.56,51.74,131.0,-0.41,0.64,3.51,XR,ATH,IT
55,RELAXO,1176.00,-38.81,47.0,X-SC,4.81,83453.0,-61707.0,137631.0,2.21,-42.51,164.92,52.31,136.0,-0.45,0.59,11.71,X40N,NTT,FOOTWEAR
6,ASIANTILES,137.00,7222.22,71.0,L-SC,6.74,80362.0,-13448.0,90062.0,2.13,-14.34,112.07,81.67,269.0,-0.15,0.57,55.63,XR,NTT,CERAMICS
81,VALIANTORG,838.00,-285.28,37.0,H-SC,6.62,102952.0,-60653.0,181968.0,2.00,-37.07,176.75,74.15,139.0,-0.33,0.73,31.97,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,BANDHANBNK,400.00,-11.26,53.0,H-SC,2.29,222266.0,-56293.0,313728.0,-0.67,-20.21,141.15,92.42,151.0,-0.18,1.58,28.45,XY24,NTT,BANKS
77,UJJIVANSFB,60.00,118.20,67.0,H-SC,13.71,137565.0,-4914.0,24432.0,-0.60,-3.45,17.76,13.70,163.0,-0.20,0.98,62.36,OX40N,NTT,BANKS
57,REPCOHOME,880.00,-56.04,66.0,H-SC,2.25,250290.0,-36215.0,293540.0,-0.42,-12.64,117.28,89.82,134.0,-0.12,1.78,32.42,XY24,NTT,FINANCE
79,UNITDSPR,1644.00,-10.66,57.0,X-LC,5.79,234075.0,-69.0,48688.0,-0.34,-0.03,20.80,20.77,86.0,-0.00,1.66,6.89,X40N,NTT,BREWERIES
33,ICICIGI,2260.25,-13.61,64.0,X-MC,2.92,148414.0,12281.0,18849.0,-0.27,9.02,12.70,22.86,91.0,0.65,1.05,22.77,X40,ATH,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.0,41.94,48.0,M-MC,10.61,224084.0,-878.0,168534.0,0.22,-0.39,75.21,74.53,192.0,-0.01,1.59,31.74,XY24,BTT,STEEL
85,WIPRO,420.0,-14.70,44.0,M-LC,6.16,149959.0,-986.0,110445.0,0.26,-0.65,73.65,72.51,53.0,-0.01,1.07,5.33,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,118.20,67.0,H-SC,13.71,137565.0,-4914.0,24432.0,-0.60,-3.45,17.76,13.70,163.0,-0.20,0.98,62.36,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.33,67.0,H-SC,2.04,232182.0,-37485.0,73811.0,1.75,-13.90,31.79,13.47,138.0,-0.51,1.65,18.06,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,95.94,49.0,M-SC,1.84,84456.0,-16321.0,16418.0,1.14,-16.20,19.44,0.09,245.0,-0.99,0.60,10.96,OX40N,NTT,DURABLES
17,CERA,9475.0,-23.39,45.0,H-SC,2.60,139874.0,-36029.0,78050.0,0.65,-20.48,55.80,23.89,149.0,-0.46,0.99,20.80,OX40N,NTT,CERAMICS
65,SIS,528.0,2078.24,53.0,H-SC,3.58,87898.0,-23134.0,46744.0,0.35,-20.84,53.18,21.26,156.0,-0.49,0.62,18.76,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.07,207515.0,4345.0,37788.0,0.69,2.14,18.21,20.74,55.0,0.11,1.47,7.80,XY25,NTT,FINANCE
31,HINDZINC,730.22,29.80,53.0,M-LC,10.40,211301.0,6225.0,106348.0,-0.13,3.04,50.33,54.89,52.0,0.06,1.50,27.86,X5K,ATH,METALS
45,KPIGREEN,731.05,7.10,67.0,H-SC,5.54,127474.0,2177.0,56751.0,1.86,1.74,44.52,47.03,141.0,0.04,0.91,59.61,MH,ATH,POWER
78,UNIONBANK,163.00,-10.08,59.0,M-LC,8.89,161261.0,20421.0,24076.0,0.35,14.50,14.93,31.59,66.0,0.85,1.15,44.83,XY24,NTT,BANKS
11,BANKINDIA,190.00,-21.74,67.0,H-MC,13.04,200786.0,20978.0,91438.0,0.60,11.67,45.54,62.52,88.0,0.23,1.43,49.73,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-37.00,70.0,M-SC,4.00,106500.0,15002.0,63751.0,4.57,16.40,59.86,86.07,223.0,0.24,0.76,54.66,XR,NTT,DURABLES
11,BANKINDIA,190.00,-21.74,67.0,H-MC,13.04,200786.0,20978.0,91438.0,0.60,11.67,45.54,62.52,88.0,0.23,1.43,49.73,XR,NTT,BANKS
31,HINDZINC,730.22,29.80,53.0,M-LC,10.40,211301.0,6225.0,106348.0,-0.13,3.04,50.33,54.89,52.0,0.06,1.50,27.86,X5K,ATH,METALS
45,KPIGREEN,731.05,7.10,67.0,H-SC,5.54,127474.0,2177.0,56751.0,1.86,1.74,44.52,47.03,141.0,0.04,0.91,59.61,MH,ATH,POWER
36,INDIAMART,4810.62,-55.01,44.0,H-SC,7.54,125631.0,2295.0,129337.0,1.53,1.86,102.95,106.72,119.0,0.02,0.89,25.67,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,15.0,X-LC,11.47,146271.0,-130252.0,240323.0,0.80,-47.10,164.30,39.81,54.0,-0.54,1.04,3.10,XY24,NTT,AUTO
51,QUESS,986.00,-47.11,38.0,X-SC,38.75,52275.0,-12731.0,163657.0,0.40,-19.58,313.07,232.18,198.0,-0.08,0.37,1.96,XY24,NTT,MISC
28,HAVELLS,2069.16,-6.46,43.0,X-MC,2.62,231956.0,-15775.0,90834.0,0.10,-6.37,39.16,30.30,92.0,-0.17,1.65,6.08,X40,ATH,ELECTRICAL
64,SIEMENS,4671.50,-3.47,43.0,H-LC,0.47,154985.0,-31110.0,78593.0,0.09,-16.72,50.71,25.51,15.0,-0.40,1.10,14.21,AR,ATH,ELECTRICAL
55,RELAXO,1176.00,-38.81,47.0,X-SC,4.81,83453.0,-61707.0,137631.0,2.21,-42.51,164.92,52.31,136.0,-0.45,0.59,11.71,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.47,43.0,H-LC,0.47,154985.0,-31110.0,78593.0,0.09,-16.72,50.71,25.51,15.0,-0.40,1.10,14.21,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,4.10,66.0,X-MC,2.03,218445.0,26703.0,11053.0,0.80,13.93,5.06,19.69,99.0,2.42,1.55,21.67,XY25,NTT,AC
41,ITC,452.00,-37.50,60.0,X-LC,2.53,202030.0,1892.0,18991.0,0.02,0.95,9.40,10.44,4.0,0.10,1.44,6.56,X40,NTT,FMCG
28,HAVELLS,2069.16,-6.46,43.0,X-MC,2.62,231956.0,-15775.0,90834.0,0.10,-6.37,39.16,30.30,92.0,-0.17,1.65,6.08,X40,ATH,ELECTRICAL
56,RELIANCE,1533.00,-8.41,74.0,X-LC,2.87,229024.0,18658.0,10123.0,0.09,8.87,4.42,13.68,37.0,1.84,1.63,26.86,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,38.0,X-SC,38.75,52275.0,-12731.0,163657.0,0.40,-19.58,313.07,232.18,198.0,-0.08,0.37,1.96,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,15.0,X-LC,11.47,146271.0,-130252.0,240323.0,0.80,-47.10,164.30,39.81,54.0,-0.54,1.04,3.10,XY24,NTT,AUTO
2,ACC,3906.00,-38.07,48.0,X-MC,3.14,184400.0,-53551.0,206196.0,0.72,-22.51,111.82,64.15,174.0,-0.26,1.31,3.44,XY24,BTT,CEMENT
21,COLPAL,3767.14,-3.45,48.0,X-MC,8.10,225260.0,-38105.0,151465.0,0.37,-14.47,67.24,43.04,84.0,-0.25,1.60,4.58,XY25,ATH,FMCG
9,BAJAJHFL,181.50,-13.63,49.0,X-MC,5.57,181778.0,-18728.0,115883.0,1.64,-9.34,63.75,48.45,90.0,-0.16,1.29,4.82,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,38.0,X-SC,38.75,52275.0,-12731.0,163657.0,0.40,-19.58,313.07,232.18,198.0,-0.08,0.37,1.96,XY24,NTT,MISC
55,RELAXO,1176.00,-38.81,47.0,X-SC,4.81,83453.0,-61707.0,137631.0,2.21,-42.51,164.92,52.31,136.0,-0.45,0.59,11.71,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.15,47.0,X-SC,4.22,96919.0,-31751.0,77051.0,0.25,-24.68,79.50,35.20,219.0,-0.41,0.69,11.24,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.38,49.0,X-SC,0.89,107475.0,5471.0,38455.0,0.27,5.36,35.78,43.06,122.0,0.14,0.76,23.40,X40N,ATH,MISC
32,HONAUT,58357.33,-22.51,50.0,X-SC,2.55,109230.0,-18708.0,65844.0,0.18,-14.62,60.28,36.84,143.0,-0.28,0.78,11.82,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-28.69,52.0,X-LC,12.66,283617.0,-62339.0,129046.0,0.07,-18.02,45.50,19.28,1.0,-0.48,2.01,4.85,X40,ATH,IT
39,INFY,2275.00,-19.24,49.0,X-LC,7.59,314026.0,1138.0,170548.0,0.92,0.36,54.31,54.87,3.0,0.01,2.23,7.04,X40,BTT,IT
41,ITC,452.00,-37.50,60.0,X-LC,2.53,202030.0,1892.0,18991.0,0.02,0.95,9.40,10.44,4.0,0.10,1.44,6.56,X40,NTT,FMCG
82,VBL,671.64,-18.54,52.0,X-LC,10.39,293289.0,-22553.0,135206.0,0.73,-7.14,46.10,35.67,5.0,-0.17,2.08,5.72,X40N,ATH,FMCG
1,ABB,7934.00,-39.77,54.0,H-LC,6.69,251904.0,-9715.0,128924.0,0.32,-3.71,51.18,45.57,7.0,-0.08,1.79,7.19,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7222.22,71.0,L-SC,6.74,80362.0,-13448.0,90062.0,2.13,-14.34,112.07,81.67,269.0,-0.15,0.57,55.63,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,54.0,L-SC,30.12,83328.0,-30221.0,70271.0,1.88,-26.61,84.33,35.27,268.0,-0.43,0.59,103.95,XR,NTT,HOTELS
48,MASFIN,398.61,-18.25,54.0,H-SC,9.49,92865.0,-5115.0,26717.0,1.99,-5.22,28.77,22.05,152.0,-0.19,0.66,34.99,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-37.00,70.0,M-SC,4.00,106500.0,15002.0,63751.0,4.57,16.40,59.86,86.07,223.0,0.24,0.76,54.66,XR,NTT,DURABLES
45,KPIGREEN,731.05,7.10,67.0,H-SC,5.54,127474.0,2177.0,56751.0,1.86,1.74,44.52,47.03,141.0,0.04,0.91,59.61,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-190.91,75.0,M-SC,21.06,156780.0,7560.0,136618.0,1.28,5.07,87.14,96.62,208.0,0.06,1.11,69.67,XY25,NTT,FINANCE
68,SURYODAY,240.0,68.86,63.0,H-SC,11.89,154020.0,-25051.0,90779.0,1.42,-13.99,58.94,36.71,135.0,-0.28,1.09,52.82,XR,NTT,BANKS
77,UJJIVANSFB,60.0,118.20,67.0,H-SC,13.71,137565.0,-4914.0,24432.0,-0.60,-3.45,17.76,13.70,163.0,-0.20,0.98,62.36,OX40N,NTT,BANKS
67,STARHEALTH,761.0,22.98,68.0,H-SC,10.41,273741.0,-7382.0,140238.0,1.64,-2.63,51.23,47.26,144.0,-0.05,1.94,46.66,XY24,NTT,INSURANCE
4,ANGELONE,3033.0,15.82,70.0,X-SC,7.68,205238.0,14232.0,43469.0,0.13,7.45,21.18,30.21,157.0,0.33,1.46,29.89,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.51
1,25,44.04
2,50,75.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.56
LC    33.74
MC    21.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.51
X40      14.75
XY25     12.37
XR       11.17
X40N     10.24
AR        7.98
OX40N     7.65
X200      1.79
X5K       1.50
SR        1.11
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.57
X-LC    22.84
X-MC    15.95
M-SC    12.84
M-LC     5.19
H-LC     4.68
X-SC     4.65
H-MC     3.83
M-MC     1.59
L-SC     1.50
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.09,-3.50,37.60
IT,12.63,-18.30,82.15
FINANCE,12.19,-10.09,59.99
MISC,6.92,-17.65,82.84
BANKS,6.54,-8.57,65.68
PAINTS,5.68,-15.38,32.58
ELECTRICAL,5.32,-10.07,48.68
INSURANCE,3.95,0.70,36.42
AUTO,2.82,-44.94,114.23


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3276838.0,22
XR,1340225.0,14
AR,1206659.0,9
X40,759491.0,10
X40N,585176.0,8
OX40N,568055.0,10
XY25,440942.0,8
SR,264722.0,2
X5K,106348.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3398945.0,25
M-SC,1234184.0,15
X-MC,1080443.0,11
X-LC,1021952.0,13
X-SC,526107.0,6
H-MC,340878.0,3
H-LC,281440.0,3
M-LC,278657.0,4
L-SC,249314.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1169110.0      6
           AR         837601.0      5
           XR         806153.0      7
M-SC       XY24       680146.0      6
X-MC       XY24       588496.0      4
X-LC       X40        468894.0      5
           XY24       304523.0      2
X-SC       X40N       285399.0      4
H-SC       SR         264722.0      2
           OX40N      264608.0      4
X-MC       X40        213546.0      4
H-LC       AR         207517.0      2
X-LC       X40N       183894.0      3
H-MC       XY24       178296.0      1
M-MC       XY24       168534.0      1
X-SC       XY24       163657.0      1
X-MC       XY25       162518.0      2
M-SC       AR         161541.0      2
L-SC       XR         160333.0      2
M-SC       OX40N      143322.0      4
           XY25       136618.0      1
X-MC       X40N       115883.0      1
M-SC       XR         112557.0      2
M-LC       XR         110445.0      1
           X5K        106348.0      1
H-MC       XR          91438.0      1
L-SC       OX40N       88981.0      1
X-SC       X40         77051.0      1
H-LC       X200        73923.0      1
H-MC       OX40N       71144.0      1
X-LC       XY25        64641.0      3
L-MC       XR          59299.0      1
H-SC       MH          56751.0      1
L-LC       XY25        39377.0      1
M-LC       XY25        37788.0      1
           XY24        24076.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
